In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

import torch

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
# E = WordEmbedding('./embeddings/GoogleNews-vectors-negative300.bin') # Not possible
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/GoogleNews-vectors-negative300.bin
(3000000, 300)
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus
Words: 3000000
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 26.777
Median: 10.0
(669425, 300)
   0                          she | he                           0.99
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                          She | He                           0.89
   4                          Her | His                          0.79
   5                          Ms. | Mr.                          0.78
   6                        woman | man                          0.75
   7                           Ms | Mr                           0.69
   8                     daughter | son                          0.67
   9                businesswoman | businessman                  0.65
  10                         girl | boy                          0.65
  11                      actress | actor                        0.65
  12                   chairwoman | chairman                     0.63
  13                      hero

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798446, 'maestro'),
  (-0.2166545, 'statesman'),
  (-0.20758688, 'skipper'),
  (-0.20267184, 'protege'),
  (-0.20206773, 'businessman'),
  (-0.1949237, 'sportsman'),
  (-0.18836349, 'philosopher'),
  (-0.18073651, 'marksman'),
  (-0.17289847, 'captain'),
  (-0.16785534, 'architect'),
  (-0.16702051, 'financier'),
  (-0.16313636, 'warrior'),
  (-0.1528085, 'major_leaguer'),
  (-0.1500145, 'trumpeter'),
  (-0.14718857, 'broadcaster'),
  (-0.14637244, 'magician'),
  (-0.14401685, 'fighter_pilot'),
  (-0.13782264, 'boss'),
  (-0.13718197, 'industrialist'),
  (-0.13684872, 'pundit')],
 [(0.1971423, 'interior_designer'),
  (0.20833445, 'housekeeper'),
  (0.21560365, 'stylist'),
  (0.22363189, 'bookkeeper'),
  (0.23776132, 'maid'),
  (0.24125952, 'nun'),
  (0.24782583, 'nanny'),
  (0.24929322, 'hairdresser'),
  (0.24946186, 'paralegal'),
  (0.25276455, 'ballerina'),
  (0.25718826, 'socialite'),
  (0.2664712, 'librarian'),
  (0.27317646, 'receptionist'),
  (0.2754029, 'waitress'),
  (0

## Find soft debias transform using SGD
This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

In [7]:
from debiaswe.debias import soft_debias

In [8]:
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)

In [9]:
soft_debias(E, gender_specific_words, defs)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 36000000000000 bytes. Error code 12 (Cannot allocate memory)


In [ ]:
# save soft-debiased embeddings

E.save('./embeddings/w2v_gnews_small_soft_debiased.txt')
# E.save('./embeddings/GoogleNews-vectors-negative300_soft_debiased.bin')

In [ ]:
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)
we.viz(a_gender_debiased)

In [ ]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]